# Implementation of Data-Driven Decision based on Causal Decision Diagram

## 1. Menyiapkan Platform dengan pustaka dan program yang dibutuhkan

In [1]:
!git clone https://github.com/alexanderkhoe/DDD.git

Cloning into 'DDD'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 18 (delta 1), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (18/18), 24.74 KiB | 8.25 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [2]:
%cd /content/DDD

/content/DDD


In [3]:
!pip install SALib
!pip install -U pymoo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.8/778.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 20.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for grapheme: filename=grapheme-0.6.0-py3-none-any.whl size=210078 sha256=392bfe1adcb405e39351ee8a785d3e1cd79d43a89acf46dc22544b2791a9e1e8
  Stored in directory: /root/.cache/pip/wheels/01/e1/49/37e6bde9886439057450c494a79b0bef8bbe897a54aebfc757
Successfully built grapheme


## 2. Membangun Causal Decision Diagram dengan mendefinisikan keputusan yang dapat diambil (Lever), variabel antara (Intermediate), hasil akhir (Outcome) dan eksternal (External).

In [4]:
from DDD.cdm import CausalDecisionModel
from DDD.element import Lever, Outcome, Intermediate, External

lever1 = Lever("Marketing Spend", 500.00, [0.00, 2000.00])
lever2 = Lever("Sales Price", 8.25, [0.04, 15.00])
lever3 = Lever("Production Order Size", 85000, [0, 200000])



In [5]:
external1 = External("Market Size", 1000, [100, 10000])


In [17]:
# Diketahui dari studi kasus: Semakin rendah harga produk, semakin tinggi persentase penjualan sebesar 11%
def f1(x): # model Price(P) bersifat linear dengan demand (Q) sebagai berikut Q = a – b*P
  return 1000-100*x

import math
def f2(x): # model Uplift (U) terhadap Marketing Spend (S) sebagai berikut U = a+b*ln(S)
  return 1000+100*math.log(x)

itm1 = Intermediate("Demand", ["Sales Price"]).fit(f1)
itm2 = Intermediate("Uplift", ["Marketing Spend"]).fit(f2)
itm3 = Intermediate("Units Sold", ["Market Size", "Uplift", "Demand"]).fit(lambda x, y, z: x*y*z)
itm4 = Intermediate("Revenue", ["Units Sold", "Sales Price"]).fit(lambda x, y: x * y)

itm5 = Intermediate("Unit Cost", ["Production Order Size"]).fit(lambda x: x)
itm6 = Intermediate("Production Cost", ["Unit Cost", "Production Order Size"]).fit(lambda x, y: x * y)
itm7 = Intermediate("Total Cost", ["Marketing Spend", "Production Cost"]).fit(lambda x, y: x + y)


In [18]:
outcome = Outcome("Profit", ["Revenue", "Total Cost"]).fit(lambda x, y: x - y)


In [19]:
model = CausalDecisionModel([lever1, lever2, lever3])
model.add_element(external1)
model.add_elements([itm1, itm2, itm3, itm4, itm5, itm6, itm7])
model.add_element(outcome)

## 3. Analisa Keputusan berdasarkan Lever

In [22]:
print(model.validate())
print(model.element("Total Cost"))
print(model.call("Profit")([1, 2, 3]))

x, y, z = model.multi_optimize(["Profit"])
print(x, y, z)



True
Intermediate(Total Cost)
4799999990.0
1999.5775731619317 4.99670359815975 198.8342747840993


In [23]:
print(model.sensitivity("Profit"))
print(model.fidelity)

(array([-0.0012428 ,  0.5662229 ,  0.43198801]), array([0.00207244, 0.56799927, 0.43156891]))
[1, 1, 1, 1, 1, 1, 1, 1]
1.0


In [24]:
print(model.optimize("Profit"))
external1.value = 3000
print(model.call("Profit")([1, 2, 3]))
print(model.optimize("Profit"))
print(model.call("Profit")([1, 2, 3]))

[1997.40644378    5.04337245  303.03127976]
4799999990.0
[1999.96899219    5.01596589   79.15195742]
4799999990.0


In [25]:
print(model.optimize("Profit"))
external1.value = 2000
print(model.call("Profit")([1, 2, 3]))
print(model.optimize("Profit"))
print(model.call("Profit")([1, 2, 3]))

[1998.4569891     5.00485738  279.28870804]
3199999990.0
[1999.94904528    5.01191685   81.29247919]
3199999990.0
